## Выбираем лучшие настройки для тикеров

In [1]:
%reload_ext autoreload
%autoreload 2

In [4]:
from bot.test import TestAlgorithm, TestAccAlgorithm
from app.cache import LocalCache
from bot.vis import TaskProgress
from app.config import RunConfig, AccConfig

test_configs = [
    {
        'acc': AccConfig.from_repr_string('t1 [0] |u0.1 d0|'),
        'bots':  [
            RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ |u0.001 d0.0| F'),
            RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤ F'),
            RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F'),
        ]
    },
    {
        'acc': AccConfig.from_repr_string('t2 [0] |u0.2 d0|'),
        'bots':  [
            RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F'),
            RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤ F'),
            RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F'),
        ]
    },
    {
        'acc': AccConfig.from_repr_string('t3 [0] |u0.3 d0|'),
        'bots':  [
            RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F'),
            RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤ F'),
            RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F'),
        ]
    },
    {
        'acc': AccConfig.from_repr_string('t5 [0] |u0.5 d0|'),
        'bots':  [
            RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F'),
            RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤ F'),
            RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F'),
        ]
    },
    {
        'acc': AccConfig.from_repr_string('t5 [0] |u0 d0|'),
        'bots':  [
            RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F'),
            RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤ F'),
            RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F'),
        ]
    },
]

def run_test(config):
    bot_alg_list = []
    for bot_cfg in config['bots']:
        bot_alg_list.append(TestAlgorithm(do_printing=False, config=bot_cfg, use_cache=True))
        
    test_acc_alg = TestAccAlgorithm(
        do_printing=False, 
        config=config['acc'],
        bot_alg_list=bot_alg_list,
    )
    
    return test_acc_alg.test(
        last_test_date='2024-10-24',
        test_days_num=30,
    )

# unique_configs = set(test_configs)
unique_configs = test_configs

results = []
LocalCache.clear()
progress = TaskProgress(len(unique_configs))

for cfg in test_configs:
    res = run_test(cfg)
    if res:
        results.append(res)
    progress.update_progress()

print(f"cache {LocalCache.get_counter('cache_miss')} vals, used {LocalCache.get_counter('cache_find')} times")

for r in results:
    print(r['acc'])
    for bot in r['bots']:
        print(f"    profit={str(bot['profit_p']) + '%':<6} op={bot['op']:<4} \"{str(bot['config'])}\"")
# print(results)


# # Вывод результатов или их дальнейшая обработка
# sorted_results = sorted(results, key=lambda x: (x['config'].ticker, x['config'].pretest_type, -float(x['profit_p'])), reverse=False)
# 
# last_val = ''
# for item in sorted_results:
#     cur_val = item['config'].ticker
#     if cur_val != last_val:
#         last_val = cur_val
#         print()
#     print(item)

Запуск в 00:09
Закончено в 00:10, количество 5, длительность 0:01:02                                               
cache 2165 vals, used 13192 times
{'profit': 0.51, 'p_avg': 0.02, 'config': t1 [0] |u0.1 d0.0| }
    profit=1.2%   op=194  "MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ |u0.001 d0.0| F "
    profit=2.16%  op=441  "RNFT+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
    profit=1.7%   op=652  "MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F "
{'profit': 1.39, 'p_avg': 0.05, 'config': t2 [0] |u0.2 d0.0| }
    profit=1.3%   op=314  "MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
    profit=2.07%  op=454  "RNFT+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
    profit=1.12%  op=682  "MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F "
{'profit': 1.75, 'p_avg': 0.06, 'config': t3 [0] |u0.3 d0.0| }
    profit=1.43%  op=327  "MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
    profit=2.24%  op=457  "RNFT+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
    profit=1.26%  op=707  "MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F "
{'profit': 1.76, 'p_avg': 

За 1 день 24 окт

    {'profit': 0.25, 'config': auto_05 [0] |u0.5 d0.0| }
        profit=0.24% op=20   "MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
        profit=0.3%  op=41   "RNFT+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
        profit=0.19% op=38   "MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F "


[[{'exp': 'MVID fan F', 'profit': 240, 'profit_p': 0.24, 'profit_p_avg': 0.24, 'config': MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F , 'last_conf': MVID+ 10/fan7:0/2 x l48 x 0.6(+x0.2)¤ F , 'op': 20}, 

{'exp': 'RNFT fan F', 'profit': 303, 'profit_p': 0.3, 'profit_p_avg': 0.3, 'config': RNFT+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F , 'last_conf': RNFT+ 8/fan7:0/2 x l40 x 0.6(+x0.2)¤ F , 'op': 41}, 

{'exp': 'MTLR fan F', 'profit': 193, 'profit_p': 0.19, 'profit_p_avg': 0.19, 'config': MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F , 'last_conf': MTLR+ 10/fan7:0/2 x l48 x 0.4¤ |u0.01 d0.0| F , 'op': 38}]]

Целевые показатели
====

длительность 0:01:00                                               
cache 1502 vals, used 1568 times

[[{'exp': 'MVID fan ', 'profit': 1049, 'profit_p': 1.05, 'profit_p_avg': 0.04, 'config': MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ , 'last_conf': MVID+ 10/fan7:0/2 x l46 x 0.6(+x0.2)¤ , 'op': 285}, 

{'exp': 'RNFT fan ', 'profit': 1930, 'profit_p': 1.93, 'profit_p_avg': 0.06, 'config': RNFT+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ , 'last_conf': RNFT+ 8/fan7:0/2 x l40 x 0.6(+x0.2)¤ , 'op': 426}, 

{'exp': 'MTLR fan ', 'profit': 131, 'profit_p': 0.13, 'profit_p_avg': 0.0, 'config': MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| , 'last_conf': MTLR+ 10/fan7:0/2 x l46 x 0.4¤ |u0.01 d0.0| , 'op': 548}]]


    {'profit': 0.53, 'p_avg': 0.02, 'config': t1 [0] |u0.1 d0.0| }
        profit=1.42%  op=320  "MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
        profit=2.21%  op=436  "RNFT+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
        profit=1.66%  op=620  "MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F "
    {'profit': 1.52, 'p_avg': 0.05, 'config': t2 [0] |u0.2 d0.0| }
        profit=1.33%  op=310  "MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
        profit=2.07%  op=454  "RNFT+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
        profit=1.12%  op=646  "MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F "
    {'profit': 1.77, 'p_avg': 0.06, 'config': t3 [0] |u0.3 d0.0| }
        profit=1.43%  op=327  "MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
        profit=2.24%  op=457  "RNFT+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
        profit=1.31%  op=665  "MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F "
    {'profit': 1.78, 'p_avg': 0.06, 'config': t5 [0] |u0.5 d0.0| }
        profit=1.48%  op=344  "MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
        profit=2.03%  op=463  "RNFT+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
        profit=1.36%  op=675  "MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F "
    {'profit': 1.78, 'p_avg': 0.06, 'config': t5 [0] |u0.5 d0.0| }
        profit=1.48%  op=344  "MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
        profit=2.03%  op=463  "RNFT+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
        profit=1.36%  op=675  "MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F "